# Pytorch中实现卷积网络：步长于填充

## 1. 特征图的尺寸：stride, padding, padding_mode

### 1.1 Stride

不知道你是否注意到，在没有其他操作的前提下，经过卷积操作之后，新生成的特征图的尺寸往往是小于上一层的特征图的。在之前的例子中，我们使用3*3的卷积核在6*6的通道上进行卷积，得到的特征图是4*4.如果在4*4的特征图上继续使用3*3的卷积核，我们得到的新特征图将是2*2尺寸。最极端的情况，我们使用1*1的卷积核，可以得到于原始通道相同的尺寸，但随着卷积神经网络的加深，特征图的尺寸是会越来越小的。

对于一个卷积神经网络而言，特征图的尺寸非常中重要，它即不能太小，也不能太大。如果特征图太小，就可能缺乏可以提取的信息，进一步缩小的可能性就更低，网络深度就会受限制。如果特征图太大，每个卷积核需要扫描的次数就越多，所需要的卷积操作就会越多，影响整体计算量。同时，卷积神经网络的图像像素量有很大的关系（记得我们之前说的吗？全连层层需要将像素拉平，每一个像素需要对应一个参数，对于尺寸$600*400$的图片需要$2.4*10^5$参数），因此，在全连接层登场之前，我们能够从特征图中提取出多少信息，并且将特征图的尺寸，也就是整体像素量缩小到什么水平，将会严重影响卷积神经网络整体的预测效果和计算性能。也因此，及时俩节特征图的大小，对于卷积神经网络的架构来说很有必要。

那么怎么找出卷积操作后的特征图的尺寸呢？假设特征图的高为$H_{out}$，特征图的宽为$W_{out}$，则对于上图所示的卷积操作，我们可以有如下式子：

![Alt text](image-105.png)

![Alt text](image-106.png)

其中，$H_{in}$与$W_{in}$是输入数据的高和宽，对于第一个卷积层而言，也就是输入图像的高和宽，对于后续的卷积层而言，就是前面层所输出的特征图的高和宽。$K_{H}$和$K_{w}$如同之前提到的，则代表在这一层与输入图像进行卷积操作的卷积核的高和宽。在之前的例子中，$H_{in}$和$W_{in}$都等于6,$K_{H}$和$K_{W}$都等于3,因此$H_{out }= W_{out}= 6-3+1 = 4$。但在实际情况中，图像的宽高往往是不一致的。因行业约定俗成，卷积核的形状往往是正方形，但理论上来说$K_{H}$和$K_{w}$也可以不一致。在PyTorch中，卷积核的大小由参数kernel_size确定。设置kernel_size = (3, 3),即表示卷积核的尺寸为（3, 3）。

这是特征图尺寸计算的“最简单”的情况。在实际进行卷积操作时，还有很多问题。比如说，现在每执行一次卷积，我们就将感受野向右移动一个像素，每扫描完一行，我们就向下移动一个像素，直到整张图片都被扫描完为止。在尺寸较小的图片上（比如，28*28像素），这样做并没有什么问题，但对于很大的图片来说（例如600*800），执行一次卷积计算就需要扫描很久，并且其中由许多像素都是被扫描了很多次的，既浪费时间又浪费资源。于是，我们定义一个新的超参数：卷积操作中的“步长”，参数名称stride(也译作步幅)。

步长是每执行完一次卷积，或扫描完一整行后，向右，向下移动的像素数。水平方向的步长管理横向移动，竖直方向的步长管理纵向移动。在PyTorch中，当我们对参数stride输入整数时，则默认调整水平方向扫描的步长。当输入数组时，则同时调整水平和竖直方向上的步长。默认情况下，水平和竖直方向的步长都是1,当我们把步长调整为（2, 2）,则每次横向和纵向移动时，都会移动2个元素。

![Alt text](image-107.png)

步长可以根据自己的需求进行调整，通常都设置为1-3之间的数字，也可以根据kernel_size来进行设置。在DNN中，我们把形如（sampels, features）结构的表数据中的列，也就是特征也叫做“维度”。对于表数据来说，要输入DNN，则需要让DNN的输入层上拥有和特征数一样的神经元，因此“高维”就意味着神经元更多。之前我们提到过，任何神经元中一个神经元上都只能有一个数字，对图像来说一个像素格子就是一个神经元，因此卷积网络中的“像素”就是最小特征单位，我们在计算机视觉中说“降维”，往往是减少一张图上的像素量。参数步长可以被用于降维，也就是可以让输入下一层的特征图像素量降低，特征图的尺寸变得更小。

以上图中的特征图为例，通道尺寸为7*7,卷积核尺寸为3*3,若没有步长，则会生成5*5的特征图（7-3+1）。但在（2, 2）的步长加持下，只会生成3*3的特征图。带步长的特征图尺寸计算公式为：

![Alt text](image-108.png)

其中S[0]代表横向步长，S[1]代表纵向的步长。步长可以加速对特征图的扫描，并加速缩小特征图，令计算更快。

### 1.2 Padding, Padding_mode

除了步长之外，还有一个常常在神经网络中出现的问题：扫描不完全或扫描不均衡。

先来看扫描不完全，同样还是7*7的特征图和3*3的卷积核：

![Alt text](image-109.png)

当步长为3时，feature map的尺寸出现了小数，无法再包含完整的像素了。在图像上来看也非常明显，当步长为3的时候，向右移动一次后，就没有足够的图像来扫描了。此时，我们不得不舍弃掉没有扫描的最后一列的像素。同时，在我们进行扫描的时候，如果我们的步长小于卷积核的宽度和长度，那部分像素就会在扫描的过程中被扫描很多次，而边缘的像素则只会在每次感受野来到边缘时被扫描到，这就会导致“中间重边缘轻”，扫描不均衡。为了解决这个问题，我们要采用“填充法”对图像进行处理。

![Alt text](image-110.png)

所谓填充法，就是在图像的两侧使用0或其他数字填上一些像素，扩大图像的面积，使得卷积核能将整个图像尽量扫描完整。

![Alt text](image-111.png)

在PyTorch中，填充与否由参数padding控制和padding_mode控制。padding接受大于0的正整数或数组作为参数，但通常我们只使用整数。padding=1则是在原通道的上下左右方向各添上1个像素，所以通道尺寸实际上会增加2*padding。padding_mode则可以控制填充什么内容。在图上展示的是zero_padding，也就是零填充，但我们也可以使用其他的填充方式。pytorch提供了两种填充方式，0填充与环形填充。在padding_mode中输入"zero"则使用0填充，输入“circular”则使用环形填充。

![Alt text](image-112.png)

![Alt text](image-113.png)

需要注意的是，虽然pytorch官方文档上所说padding_mode可以接受四种填充模式，但实际上截至版本1.7.1,仍然只有“zeros”和“circular”两种模式有效，其他输入都会被当成0填充。如果想要使用填充镜面翻转值的reflection padding，则必须使用单独定义的层nn.ReflectionPad2d，同样的，"replicate"模式所指代的填充边缘重复值需要使用单独的类nn.ReplicationPad2d。

![Alt text](image-114.png)

不难发现，如果输入通道的尺寸较小，padding数目又很大，padding就可能极大地扩充通道的尺寸，并让feature map在同样的卷积核下变得更大。我们之前说，在没有其他操作时feature map往往是小于输入通道的尺寸的，而加入padding之后feature map就有可能大于输入通道了，这在经典卷积网络的架构中也曾出现过。通常来说，我们还是会让feature map随着卷积层的深入逐渐变小，这样模型计算才会更快，因此，padding的值也不会很大，基本只在1-3之间。

实际上，Padding并不能够保证图像一定被扫描完全或一定均衡。看下面的例子：

![Alt text](image-115.png)

不难发现，即便已经填充了一个像素，在现在的步长与卷积核大小小，依然无法将整张图扫描完全。此时，有两种解决方案，一种叫做“Valid”,一种叫做“same”。

![Alt text](image-116.png)

valid模式就是放弃治疗，对于扫描不到的部分，直接丢弃。“same”模式是指，在当前卷积层与步长设置下无法对全图进行扫描时，对图像右侧和下边进行“再次填充”，直到扫描被允许进行为止。从上图看，same模式下的padding设置本来时1（即左右两侧都填上0），但在右侧出现11,12,13和填充的0列无法被扫描的情况，则神经网络自动按照kernel_size的需要，在右侧再次填充一个0列，实现再一次扫描，让全部像素都被扫描到。

这个操作看上去很只能，但遗憾的是只能够再tensorflow中实现，对于Pytorch而言，没有“same”的选项，只要无法扫描完全，一律抛弃。为什么这样做呢？主要还是因为kernel_size的值一般比较小，所以被漏掉的像素点不会很多，而且基本集中在边缘。随着计算机视觉中所使用的图片分辨率越来越高，图像尺寸越来越大，边缘像素包含关键信息的可能性会越来越小，丢弃边缘就变得月来越经济。对于一张28*28的图像而言，丢弃2,3列或许会有不少信息损失，但对于720*1080的图片而言，究竟是720*1078还是720*1080,其实并无太大区别。

那如果，你的图像尺寸确实较小，你希望尽量避免未扫描的像素被丢弃，那你可以如下设置：

1. 卷积核尺寸控制在5*5以下，并且kernel_size > stride
2. 令2*padding > stride

这样做不能100%避免风险，但可以大规模降低像素被丢弃的风险（个人经验，无理论基础）。

padding操作会影响通道的大小，因此padding也会改变feature map的尺寸，当padding中输入的值为P时，特征图的大小具体如下：

![Alt text](image-117.png)

我们在代码中来感受一下特征图尺寸的变化。

In [1]:
import torch
from torch import nn
# 计算公式
# (w + 2p -k)/s + 1
# (h + 2p -k)/s + 1
# 当我们不调整conv2d中的参数时，P默认为0，S默认为1

data = torch.ones(10, 3, 28, 28)
conv1 = nn.Conv2d(3, 6, 3)
conv2 = nn.Conv2d(6, 10, 3)
conv3 = nn.Conv2d(10, 16, 5, stride=2, padding=1)
conv4 = nn.Conv2d(16, 32, 3, stride=3, padding=2)

# 传入数据
output1 = conv1(data)
output1.shape

torch.Size([10, 6, 26, 26])

In [2]:
output2 = conv2(output1)
output2.shape

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

torch.Size([10, 10, 24, 24])

In [3]:
output3 = conv3(output2)
output3.shape

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

torch.Size([10, 16, 11, 11])

In [4]:
output4 = conv4(output3)
output4.shape

torch.Size([10, 32, 5, 5])